In [ ]:
!git clone https://github.com/awslabs/mlm-scoring.git
!pip install -e mlm-scoring/
!pip install torch mxnet-cu101mkl

Cloning into 'mlm-scoring'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 163 (delta 0), reused 3 (delta 0), pack-reused 156
Receiving objects: 100% (163/163), 23.50 MiB | 12.05 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mlm-scoring
     |████████████████████████████████| 236 kB 9.1 MB/s 
     |████████████████████████████████| 92 kB 12.5 MB/s 
     |████████████████████████████████| 1.1 MB 71.7 MB/s 
     |████████████████████████████████| 3.0 MB 57.4 MB/s 
     |████████████████████████████████| 1.2 MB 65.7 MB/s 
     |████████████████████████████████| 880 kB 72.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.8.3-py3-none-any.whl size=293554 sha256=b8e3b9ebf58d37417eca2650aa794067c96eebb88854cc02467f595d51a4b78e
  Stored in directory: /root/.ca

In [ ]:
%cd /content/mlm-scoring/src
from google.colab import drive
drive.mount('/content/gdrive/')
WORK_PATH = "/content/gdrive/My Drive/TMaze"
from mlm.scorers import MLMScorer, MLMScorerPT, LMScorer 
from mlm.models import get_pretrained
import mxnet as mx
import pickle
import string
from scipy.stats import norm
import spacy
import numpy as np #shouldn't need to import these
import pandas as pd
import sys
sys.path.append(WORK_PATH)
import tmaze #but can't import from within this file for some reason???? need setuptools

#debugging libraries 
import copy
import time

[Errno 2] No such file or directory: '/content/mlm-scoring/src'
/content
Mounted at /content/gdrive/


ModuleNotFoundError: ignored

In [ ]:
ctxs = [mx.gpu(0)]
#Uncomment following line If CPU only
#ctxs = [mx.cpu()]
model, vocab, tokenizer = get_pretrained(ctxs, 'bert-base-multi-cased') 
#hopefully this model is good enough, even though not exclusively trained on German - what's the best German model???
#!python -m spacy download de_core_news_sm
#nlp = spacy.load('de_core_news_sm')

In [ ]:
scorer = MLMScorer(model, vocab, tokenizer, ctxs)
print(scorer.score_sentences(["Ich will es ausprobieren."]))

[-20.469610914587975]


In [ ]:
#Making sure that these are lower - yes, second is worse than the first, not sure whether this makes sense
print(scorer.score_sentences(["Ich will ich ausprobieren."])) 
print(scorer.score_sentences(["Ich will Katze ausprobieren."])) 

In [ ]:
pickle_dict = {"freq_dict":f"{WORK_PATH}/FreqToWords.pkl", "word_info": f"{WORK_PATH}/WordInfo.pkl", "nonwords_set": f"{WORK_PATH}/NonwordsSet.pkl",
               "dists_dict": f"{WORK_PATH}/PotentialDistractors.pkl"}

In [ ]:
empty_dict = {}
with open(pickle_dict['dists_dict'], "wb") as f:
  pickle.dump(empty_dict,f)

In [ ]:
test = tmaze.tmaze(scorer,'de_core_news_sm',WORK_PATH,pickle_dict)

In [ ]:
with open(f'{WORK_PATH}/FormattedTestItems.txt') as f: #combined fillers and test sentences 
    sents = f.readlines() 

In [ ]:
#Create separate lists for the experimental and filler sentences 
exp_sent_lists = []
filler_sent_lists = []
for sent in sents:
  sent_info = sent.split(";")
  if sent_info[0] == "filler":
    filler_sent_lists+=[sent_info[2].split()]
  else:
    exp_sent_lists+=[sent_info[2].split()]
exp_sent_lists_copy = copy.deepcopy(exp_sent_lists)
filler_sent_lists_copy = copy.deepcopy(filler_sent_lists)

In [ ]:
print(len(filler_sent_lists))
print(len(exp_sent_lists))

99
90


In [ ]:
with open(f'{WORK_PATH}/FreqToWords.pkl','rb') as f:
    freq_dict_copy = pickle.load(f)

In [ ]:
with open(f'{WORK_PATH}/WordInfo.pkl','rb') as f: 
    word_info_copy = pickle.load(f)

In [ ]:
with open(f"{WORK_PATH}/NonwordsSet.pkl", "rb") as f:
    nonwords_set_copy = pickle.load(f)

In [ ]:
test_sent = sents[0].split(";")[2].split()
test_sent_copy = test_sent[:]

In [ ]:
#case 1 test
distractor = "xxx"
print(test.create_sent(0,distractor,test_sent)) 
#case 2 test
print(test.create_sent(len(test_sent)-1,distractor,test_sent))
#middle of sentence punct case
print(test.create_sent(11,distractor,test_sent))
#general case
inds = np.random.randint(1,len(test_sent)-1,3)
for i in inds:
  print(test.create_sent(i,distractor,test_sent))
print(test_sent == test_sent_copy)

xxx Mutter von Paula und die Schwester von Sophie grüßten den Direktor, den Maria und Franziska ignoriert hatten.
Die Mutter von Paula und die Schwester von Sophie grüßten den Direktor, den Maria und Franziska ignoriert xxx.
Die Mutter von Paula und die Schwester von Sophie grüßten den xxx, den Maria und Franziska ignoriert hatten.
Die Mutter von Paula und die Schwester von Sophie grüßten den Direktor, den Maria und Franziska xxx hatten.
Die Mutter von Paula und die Schwester von Sophie grüßten den Direktor, den Maria und xxx ignoriert hatten.
Die Mutter von Paula xxx die Schwester von Sophie grüßten den Direktor, den Maria und Franziska ignoriert hatten.
True


In [ ]:
#testing just_preceding parameter
print(test.create_sent(1,distractor,test_sent,True)) 
#case 2 test
print(test.create_sent(len(test_sent)-1,distractor,test_sent,True))
#middle of sentence punct case
print(test.create_sent(11,distractor,test_sent,True))
#general case
inds = np.random.randint(1,len(test_sent)-1,3)
for i in inds:
  print(test.create_sent(i,distractor,test_sent,True))
print(test_sent == test_sent_copy)

Die xxx
Die Mutter von Paula und die Schwester von Sophie grüßten den Direktor, den Maria und Franziska ignoriert xxx.
Die Mutter von Paula und die Schwester von Sophie grüßten den xxx,
Die Mutter von Paula xxx
Die Mutter von Paula und die Schwester von Sophie grüßten xxx
Die Mutter von Paula und die Schwester von Sophie grüßten den xxx,
True


In [ ]:
for i in [3,5,10]:
  print(test.init_best_list(i))

[(0, ''), (0, ''), (0, '')]
[(0, ''), (0, ''), (0, ''), (0, ''), (0, '')]
[(0, ''), (0, ''), (0, ''), (0, ''), (0, ''), (0, ''), (0, ''), (0, ''), (0, ''), (0, '')]


In [ ]:
dists = ["hallo","ich","mag","pizza"]
print(test.any_tagged(dists))
dists = ["hallo", ("ich","PRON"), ("mag","VVFIN"),"pizza"]
print(test.any_tagged(dists))
dists = [("ich","PRON"), ("mag","VVFIN")]
print(test.any_tagged(dists)) 

(False, False)
(False, True)
(True, True)


In [ ]:
#PLLs change with each restart even with same context
#TODO: do more thorough testing/printing
#Matches what's going on in other notebook pretty well though
dists = ["hallo","ich","mag","pizza"]
dists_copy = dists[:]
best_list = test.init_best_list(3)
print(test.eval_best(2,dists,test_sent,best_list))
test_ind = 6 #arbitrary - feel free to change
best_list = test.init_best_list(3)
best_so_far,evaluated = test.eval_best(test_ind,dists,test_sent,best_list) 
print((best_so_far,evaluated))
print(dists == dists_copy)
new_dists = ["gut","das","freut","mich"]
new_dists_copy = new_dists[:]
print(test.eval_best(test_ind,new_dists,test_sent,best_so_far))
print(test.eval_best(test_ind,new_dists,test_sent,best_so_far,just_preceding=True)) #should give different PLLs
print(new_dists == new_dists_copy)
new_dists = [("du","PRON"),("magst","VVFIN"),("pizza","NE"), ("auch","ADV")]
new_dists_copy = new_dists[:]
print(test.eval_best(test_ind,new_dists,test_sent,best_so_far,True))
print(new_dists == new_dists_copy)
#NOTE: tagged and untagged can be mixed - not that this is an anticipated use case

([(-73.29378584303777, 'pizza'), (-75.34832774451934, 'hallo'), (-77.28522683499614, 'mag')], {'mag', 'hallo', 'ich', 'pizza'})
([(-64.17616746784188, 'ich'), (-69.6569750359049, 'mag'), (-72.00143465620931, 'hallo')], {'mag', 'hallo', 'ich', 'pizza'})
True


/usr/local/lib/python3.7/dist-packages/gluonnlp/data/sampler.py:354: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[29]
  str(unused_bucket_keys))


([(-69.6569750359049, 'mag'), (-72.00143465620931, 'hallo'), (-72.09987840690883, 'das')], {'freut', 'gut', 'mich', 'das'})
([(-68.51866429304937, 'gut'), (-69.6569750359049, 'mag'), (-72.00143465620931, 'hallo')], {'freut', 'gut', 'mich', 'das'})
True


/usr/local/lib/python3.7/dist-packages/gluonnlp/data/sampler.py:354: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[10]
  str(unused_bucket_keys))


([(-68.51866429304937, 'gut'), (-69.6569750359049, 'mag'), (-72.00143465620931, 'hallo')], {'du', 'magst', 'pizza', 'auch'})
True


In [ ]:
#test normal distribution way of getting good length range for distractors
#all with 20% of normal curve included in the range 
print(test.get_len_range(9))
print(test.get_len_range(7))
print(test.get_len_range(5))
print(test.get_len_range(3))
print(test.get_len_range(12))
#now with different percentages
print(test.get_len_range(7,0.3))
print(test.get_len_range(7,0.1))
print(test.get_len_range(7,0.45))
print(test.get_len_range(7,0.6))
print(test.get_len_range(7,0.85))

[8, 9]
[6, 7]
[2, 6]
[2, 5]
[10, 13]
[5, 7]
[6, 7]
[5, 8]
[3, 8]
[2, 9]


In [ ]:
dists = ["gut","das","freut","mich"]
dists_copy = dists[:]
inds = np.random.randint(1,len(test_sent),4)
for i in range(len(dists)):
  if i > 1:
    print(f'{test.create_sent(inds[i],dists[i],test_sent,True)}:{test.get_tag(dists[i],inds[i],test_sent,True)}')
  else:
    print(f'{test.create_sent(inds[i],dists[i],test_sent,True)}:{test.get_tag(dists[i],inds[i],test_sent,False)}')
print(dists == dists_copy)
filler_sent = filler_sent_lists[57] 
for i in range(len(filler_sent)):
  if filler_sent[i][-1] in string.punctuation:
    dist = filler_sent[i][:-1] #distractor won't come with punctuation
  else:
    dist = filler_sent[i]
  print(f'{test.create_sent(i,dist,filler_sent,True)}:{test.get_tag(dist,i,filler_sent,True)}')
dist = "nizza" 
print(f'{test.create_sent(6,dist,filler_sent,True)}:{test.get_tag(dist,6,filler_sent,True)}')
dist = "wie"
print(f'{test.create_sent(6,dist,filler_sent,True)}:{test.get_tag(dist,6,filler_sent,True)}')

Die Mutter gut:ADJD
Die Mutter von Paula und die das:ART
Die Mutter von freut:NE
Die Mutter von Paula und die mich:PPER
True
Die:ART
Die neuen:ADJA
Die neuen Biersorten:NN
Die neuen Biersorten der:ART
Die neuen Biersorten der Brauerei:NN
Die neuen Biersorten der Brauerei enthalten:ADJD
Die neuen Biersorten der Brauerei enthalten 6%:CARD
Die neuen Biersorten der Brauerei enthalten 6% Alkohol,:NN
Die neuen Biersorten der Brauerei enthalten 6% Alkohol, was:PWS
Die neuen Biersorten der Brauerei enthalten 6% Alkohol, was für:APPR
Die neuen Biersorten der Brauerei enthalten 6% Alkohol, was für mich:PPER
Die neuen Biersorten der Brauerei enthalten 6% Alkohol, was für mich etwas:PIS
Die neuen Biersorten der Brauerei enthalten 6% Alkohol, was für mich etwas zu:PTKZU
Die neuen Biersorten der Brauerei enthalten 6% Alkohol, was für mich etwas zu viel:PIS
Die neuen Biersorten der Brauerei enthalten 6% Alkohol, was für mich etwas zu viel ist.:VAFIN
Die neuen Biersorten der Brauerei enthalten nizza%:

In [ ]:
def display_context(test_sent,ind,just_preceding=False):
  sent = ""
  for i in range(len(test_sent)):
    if i==ind:
      sent+="___"
    elif i < ind or (i > ind and not just_preceding):
      sent+=test_sent[i]
    sent+=" "
  print(sent[:-1])

#just default paramaters
dists = ["ich", "habe", "viel", "hunger"]
dists_copy = dists[:]
ind = np.random.randint(1,len(test_sent))
display_context(test_sent,ind)
print(test.pos_shennanigans(dists,ind,test_sent))
print(dists == dists_copy)

#flip tagged parameter
dists = ["wie", "fuhlst", "du", "dich"]
dists_copy = dists[:]
ind = np.random.randint(1,len(test_sent))
display_context(test_sent,ind)
#should just return the distractors but properly capitalized
print(test.pos_shennanigans(dists,ind,test_sent,tagged=False)) 
print(dists == dists_copy)

#TODO: FIX THIS TEST
#add compare_tag 
dists = ["sie", "will", "frühstuck", "essen"]
dists_copy = dists[:]
ind = np.random.randint(1,len(test_sent))
sent = f"{test_sent[:ind]} ___ {test_sent[ind+1:]}"
word_to_replace = test_sent[ind]
tag = test.get_tag(word_to_replace, ind, test_sent, False)
display_context(test_sent,ind)
print(f'{word_to_replace},{tag}')
#doesn't capitalize Frühstuck because tagging isn't a solved problem
#print(test.pos_shennanigans(dists,ind,test_sent,{"Frühstuck"},compare_tag=tag)) 
#print(dists == dists_copy)

#flip just_preceding parameter
dists = ["wir", "können", "alle", "zusammen", "gehen"]
dists_copy = dists[:]
ind = np.random.randint(1,len(test_sent))
display_context(test_sent,ind,True)
print(test.pos_shennanigans(dists,ind,test_sent,just_preceding=True))
print(dists == dists_copy)

#failing on this in later test case
sent_ind = 51
sent = filler_sent_lists[sent_ind]
for i in range(1, len(sent)):
  display_context(sent,i,True)
  print(test.pos_shennanigans(dists,i,sent, just_preceding=True))
print(dists == dists_copy)
print(test_sent == test_sent_copy)
print(test.nonwords_set == nonwords_set_copy)

Die Mutter von Paula und ___ Schwester von Sophie grüßten den Direktor, den Maria und Franziska ignoriert hatten.
[('ich', 'PPER'), ('habe', 'VAFIN'), ('viel', 'PIAT'), ('hunger', 'ADJA')]
True
Die Mutter von Paula und die Schwester ___ Sophie grüßten den Direktor, den Maria und Franziska ignoriert hatten.
['wie', 'fuhlst', 'Du', 'dich']
True
Die Mutter von Paula und die Schwester von Sophie grüßten den Direktor, den Maria und ___ ignoriert hatten.
Franziska,NE
Die Mutter von Paula und ___            
[('wir', 'PPER'), ('können', 'VMFIN'), ('alle', 'PIAT'), ('zusammen', 'ADV'), ('gehen', 'VVINF')]
True
Die ___                
[('wir', 'PPER'), ('können', 'VMFIN'), ('alle', 'PIS'), ('zusammen', 'ADV'), ('gehen', 'VVFIN')]
Die Pakete, ___               
[('wir', 'PPER'), ('können', 'VMFIN'), ('alle', 'PIS'), ('zusammen', 'ADV'), ('gehen', 'VVFIN')]
Die Pakete, die ___              
[('wir', 'PPER'), ('können', 'VMFIN'), ('alle', 'PIS'), ('zusammen', 'ADV'), ('gehen', 'VVFIN')]
Die Pakete

In [ ]:
init_params = [8.7,0.2]
temp_freqs = [6.7,8.7]
targets = (6,7.7)
params_copy = init_params[:]
temp_freqs_copy = temp_freqs[:]
targets_copy = targets #tuples are imutable
#case where we're about to go over
print(test.adjust_params(init_params,temp_freqs,targets,True)) 
print(init_params == params_copy)
print(temp_freqs == temp_freqs_copy)
print(targets == targets_copy)

#changes its parameters - intended
temp_freqs = [6.9,7.3]
targets = (6,7.1)
temp_freqs_copy = temp_freqs[:]
targets_copy = targets
params, new_temp_freqs = test.adjust_params(init_params,temp_freqs,targets,True) #just increase temp_freqs
print(params,new_temp_freqs)
print(init_params == params_copy and params_copy == params)
print(temp_freqs == temp_freqs_copy)
print(targets == targets_copy)

#the floating point errors are corrected right before checking the dictionary for the values
print(test.adjust_params(params,new_temp_freqs,targets,False)) #nothing should happen because we need to keep working through that bin
print(init_params == params_copy and params_copy == params)
print(temp_freqs == temp_freqs_copy)
print(targets == targets_copy)

([8.799999999999999, 0.22], [7.7, 7.7])
True
True
True
[8.7, 0.2] [6.890000000000001, 7.31]
True
True
True
([8.7, 0.2], [6.890000000000001, 7.31])
True
True
True


In [ ]:
dist = "Bäckerin"
test.get_rating(dist,1,test_sent)
test.get_rating(dist,12,test_sent)
test.get_rating(dist,7,test_sent)
print(test_sent == test_sent_copy)

Die Bäckerin [Mutter]
Unacceptable? 0
Die Mutter von Paula und die Schwester von Sophie grüßten den Direktor, Bäckerin [den]
Unacceptable? 1
Die Mutter von Paula und die Schwester Bäckerin [von]
Unacceptable? 0
True


In [ ]:
#Some dummy test cases
dummy_dist_dict = {} #could be an object itself
new_dists = ["yo","sup","does","this","work"]
word = "word"
test.add_to_distractor_dict(word,new_dists,dummy_dist_dict)
print(dummy_dist_dict)
test.dists_dict = dummy_dist_dict
print(test.check_for_potential_distractors("new",100))
print(test.check_for_potential_distractors(word,5))
print(test.check_for_potential_distractors(word,100))
more_dists = ["time","to","add","some","more"]
test.add_to_distractor_dict(word,more_dists,dummy_dist_dict)
print(test.check_for_potential_distractors(word,5,))
print(test.dists_dict)
#reset dists_dict to its original
with open(pickle_dict["dists_dict"], "rb") as f:
  test.dists_dict = pickle.load(f) 

{'word': {'work', 'does', 'this', 'yo', 'sup'}}
([], 0)
(['work', 'does', 'this', 'yo', 'sup'], 5)
(['work', 'does', 'this', 'yo', 'sup'], 5)
(['work', 'does', 'this', 'time', 'more', 'yo'], 5)
{'word': {'work', 'does', 'this', 'time', 'more', 'yo', 'to', 'sup', 'some', 'add'}}


In [ ]:
test.pickle_dict

{'dists_dict': '/content/gdrive/My Drive/TMaze/PotentialDistractors.pkl',
 'freq_dict': '/content/gdrive/My Drive/TMaze/FreqToWords.pkl',
 'nonwords_set': '/content/gdrive/My Drive/TMaze/NonwordsSet.pkl',
 'word_info': '/content/gdrive/My Drive/TMaze/WordInfo.pkl'}

Parameters to test:
* tagged ✅
* use_tag ✅
* filler ✅
* just_preceding ✅
* rate ✅
* dist_df ✅
* dist_csv ✅
* save_csv
  * boolean ✅
  * string ✅
* multi ✅
* verbose ✅


In [ ]:
#Check that we won't save duplicates
#Testing use_tag annd verbose
sent_ind = 80
test_sent = exp_sent_lists[sent_ind]
test_sent_copy = test_sent[:]
dist_ind = 1
print(test.batch_eval(dist_ind,test_sent,sent_ind,10,3,use_tag=True,verbose=True)) 
#Only predetermine the tag in the case of use_tag
print("All the following should be true if we're not mutating anything that we shouln't be.")
print(test_sent == test_sent_copy)
print(test.nonwords_set == nonwords_set_copy)
print(test.word_info == word_info_copy)
print(test.freq_dict == freq_dict_copy)
print(len(test.dists_dict)) #too big to print after having used it for many rounds

sent_ind = 74
test_sent = exp_sent_lists[sent_ind]
test_sent_copy = test_sent[:]
dist_ind = 1
print(test.batch_eval(dist_ind,test_sent,sent_ind,10,3))
print("All the following should be true if we're not mutating anything that we shouldn't be.")
print(test_sent == test_sent_copy)
print(test.nonwords_set == nonwords_set_copy)
print(test.word_info == word_info_copy)
print(test.freq_dict == freq_dict_copy)
print(len(test.dists_dict)) 

Word: und, Tag: KON
7.42
Time to evaluate >10 distractors: 0.21800518035888672
Final parameter settings: 7.52 0.2
Vanessa laden [und]
Unacceptable? 1
Vanessa Tage [und]
Unacceptable? 0
Vanessa wovon [und]
Unacceptable? 1
/content/gdrive/My Drive/TMaze/PotentialDistractors.pkl
  Distractor Index    POS Sent Index Filler Replaced_Word Replaced_POS  \
0      laden     1  VVINF         80  False           und          KON   
1       Tage     1     NN         80  False           und          KON   
2      wovon     1   PWAV         80  False           und          KON   

         PLL  PLL norm Number Evaluated Best N Ranking Rating  Evaluation Time  
0 -30.130128 -6.026026               10      3       3      1         0.218005  
1 -31.504324 -7.876081               10      3       2      0         0.218005  
2 -35.829246 -7.165849               10      3       1      1         0.218005  
All the following should be true if we're not mutating anything that we shouln't be.
True
True
True
Tr

In [ ]:
#Testing save_top_distractors through batch_eval
sent_ind = np.random.randint(0,90)
test_sent = exp_sent_lists[sent_ind]
dist_ind = 1
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3)
#just reevaluating with more distractors - shouldn't make us rate the distractors again even without the multi parameter
print("Some of these probably won't be new...")
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,20,3,dist_df=new_df)
#appending but with a different sentence this time
sent_ind = np.random.randint(0,90)
test_sent = exp_sent_lists[sent_ind]
#rate set to false
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,dist_df=new_df,rate=False)
print(new_df)
#Saving in a csv
dist_ind = 2
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,dist_df=new_df,save_csv=True)
print(new_df)
dist_ind = 3 
dumb_test_name = "ToTestNamingDistractorFile.csv"
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,dist_df=new_df,save_csv=dumb_test_name)
print(new_df)
#Appending to data in csv
dist_ind = 4
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,dist_csv="DistractorRecord.csv")
print(new_df)
dist_ind = 5
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,dist_csv=dumb_test_name)
print(new_df)
#Tagged set to false
print(test.batch_eval(dist_ind,test_sent,sent_ind,10,3,tagged=False))
#Pass a filler
dist_ind = 1
test_sent = filler_sent_lists[sent_ind]
new_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,filler=True,dist_df=new_df)
print(new_df)
#just_preceding set to false
#the same distractors with the same PLLS and wrong POS tags for both - suggests that this doesn't work
print(test.batch_eval(dist_ind,test_sent,sent_ind,10,3,filler=True,just_preceding=False,dist_df=new_df))

print(len(test.dists_dict))
print("All the following should be true if we're not mutating anything that we shouldn't be.")
#print(test_sent == test_sent_copy) #actually this one should be false because we've since changed the test_sent and not copied it hehe
print(test.nonwords_set == nonwords_set_copy)
print(test.word_info == word_info_copy)
print(test.freq_dict == freq_dict_copy)

7.42
Time to evaluate >10 distractors: 0.20922303199768066
Final parameter settings: 7.52 0.2
Sonja nähe [und]
Unacceptable? 1
Sonja raum [und]
Unacceptable? 1
Sonja voran [und]
Unacceptable? 1
Some of these probably won't be new...
7.42
Time to evaluate >20 distractors: 0.21239924430847168
Final parameter settings: 7.52 0.2
Sonja gutem [und]
Unacceptable? 1
Sonja weile [und]
Unacceptable? 1
Sonja wüste [und]
Unacceptable? 0
7.42
Time to evaluate >10 distractors: 0.21112370491027832
Final parameter settings: 7.52 0.2
  Distractor Index    POS Sent Index Filler Replaced_Word Replaced_POS  \
0       nähe     1  VVFIN         73  False           und          KON   
1       raum     1  VVFIN         73  False           und          KON   
2      voran     1  PTKVZ         73  False           und          KON   
3      gutem     1  VVFIN         73  False           und          KON   
4      weile     1  VVFIN         73  False           und          KON   
5      wüste     1  VVFIN        

In [ ]:
#add_rating testing on already existing 
new_df = test.add_ratings(new_df,exp_sent_lists,filler_sent_lists)
print(new_df)
print(exp_sent_lists == exp_sent_lists_copy)
print(filler_sent_lists == filler_sent_lists_copy)

#when adding straight to a dataframe not coming from a csv
sent_ind = np.random.randint(0,90)
test_sent = exp_sent_lists[sent_ind]
newer_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,rate=False)
print(newer_df)
newer_df = test.add_ratings(newer_df,exp_sent_lists,filler_sent_lists)
print(newer_df)
print(exp_sent_lists == exp_sent_lists_copy)
print(filler_sent_lists == filler_sent_lists_copy)

#adding to a saved csv
sent_ind = np.random.randint(0,90)
test_sent = exp_sent_lists[sent_ind]
newest_df = test.batch_eval(dist_ind,test_sent,sent_ind,10,3,rate=False,save_csv=dumb_test_name)
print(test.add_ratings(newest_df,exp_sent_lists,filler_sent_lists))
print(exp_sent_lists == exp_sent_lists_copy)
print(filler_sent_lists == filler_sent_lists_copy)

#nothing should happen here
print(test.add_ratings(new_df,exp_sent_lists,filler_sent_lists))
print(exp_sent_lists == exp_sent_lists_copy)
print(filler_sent_lists == filler_sent_lists_copy)

Lea nähe [und]
Unacceptable? 1
Lea hohe [und]
Unacceptable? 1
Lea raum [und]
Unacceptable? 1
   Distractor  Index    POS  Sent Index  Filler Replaced_Word Replaced_POS  \
0        nähe      1  VVFIN          73   False           und          KON   
1        raum      1  VVFIN          73   False           und          KON   
2       voran      1  PTKVZ          73   False           und          KON   
3       gutem      1  VVFIN          73   False           und          KON   
4       weile      1  VVFIN          73   False           und          KON   
5       wüste      1  VVFIN          73   False           und          KON   
6        nähe      1  VVFIN          52   False           und          KON   
7        hohe      1  VVFIN          52   False           und          KON   
8        raum      1  VVFIN          52   False           und          KON   
9        nähe      2  VVFIN          52   False           die          ART   
10        can      2     XY          52   False  

Parameters to test:
* start_ind
  * valid value
  * invalid value
* end_ind 
  * valid value
  * invalid value

Plugged into batch_eval and therefore already tested:
* tagged
* use_tag
* filler
* just_preceding
* rate
* dist_df
* dist_csv


In [ ]:
sent_ind = 0 #a nice and short sentence
fill_sent = filler_sent_lists[sent_ind]
fill_sent_copy = fill_sent[:]
sent_len = len(fill_sent)
print(sent_len)
ind_list = [(-3,6),(3,sent_len+2), (2,5), (5,3)]
batches = [10,20,30]
#default start_ind and end_ind - just do the whole sentence
print(test.compare_batches(batches,fill_sent,sent_ind,3,filler=True))
print(fill_sent == fill_sent_copy)

9
Round 1/24: Find distractor for word at index 1 by comparing 10 potential distractors
3.71
Time to evaluate >10 distractors: 0.6634187698364258
Final parameter settings: 3.81 0.2
Thomas überdenken [beobachtete]
Unacceptable? 1
Thomas instrumenten [beobachtete]
Unacceptable? 1
Thomas festzuhalten [beobachtete]
Unacceptable? 0
Round 2/24: Find distractor for word at index 1 by comparing 20 potential distractors
3.71
Time to evaluate >20 distractors: 0.21275734901428223
Final parameter settings: 3.81 0.2
Distractor instrumenten already rated 1
Thomas medikamenten [beobachtete]
Unacceptable? 1
Thomas radfahren [beobachtete]
Unacceptable? 0
Round 3/24: Find distractor for word at index 1 by comparing 30 potential distractors
3.71
Time to evaluate >30 distractors: 0.3139307498931885
Final parameter settings: 3.81 0.2
Distractor festzuhalten already rated 0
Distractor medikamenten already rated 1
Distractor radfahren already rated 0
Round 4/24: Find distractor for word at index 2 by compari

/usr/local/lib/python3.7/dist-packages/gluonnlp/data/sampler.py:354: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[8]
  str(unused_bucket_keys))


Time to evaluate >10 distractors: 0.24312782287597656
Final parameter settings: 4.71 0.2
Thomas beobachtete seinen Analyse [Nachbarn]
Unacceptable? 1
Thomas beobachtete seinen getötet [Nachbarn]
Unacceptable? 1
Thomas beobachtete seinen aussagen [Nachbarn]
Unacceptable? 1
Round 8/24: Find distractor for word at index 3 by comparing 20 potential distractors
4.61
Time to evaluate >20 distractors: 0.31321024894714355
Final parameter settings: 4.71 0.2
Distractor aussagen already rated 1
Thomas beobachtete seinen erfüllt [Nachbarn]
Unacceptable? 1
Thomas beobachtete seinen Weshalb [Nachbarn]
Unacceptable? 1
Round 9/24: Find distractor for word at index 3 by comparing 30 potential distractors
4.61


/usr/local/lib/python3.7/dist-packages/gluonnlp/data/sampler.py:354: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[8, 9]
  str(unused_bucket_keys))


Time to evaluate >30 distractors: 0.6227614879608154
Final parameter settings: 4.71 0.2
Distractor aussagen already rated 1
Distractor erfüllt already rated 1
Distractor Weshalb already rated 1
Round 10/24: Find distractor for word at index 4 by comparing 10 potential distractors
5.11
Time to evaluate >10 distractors: 0.4307096004486084
Final parameter settings: 5.21 0.2
Thomas beobachtete seinen Nachbarn august, [Peter,]
Unacceptable? 1
Thomas beobachtete seinen Nachbarn mitte, [Peter,]
Unacceptable? 1
Thomas beobachtete seinen Nachbarn Raum, [Peter,]
Unacceptable? 1
Round 11/24: Find distractor for word at index 4 by comparing 20 potential distractors
5.11
Time to evaluate >20 distractors: 0.6459290981292725
Final parameter settings: 5.21 0.2
Thomas beobachtete seinen Nachbarn dachte, [Peter,]
Unacceptable? 1
Thomas beobachtete seinen Nachbarn mach, [Peter,]
Unacceptable? 1
Thomas beobachtete seinen Nachbarn setzen, [Peter,]
Unacceptable? 1
Round 12/24: Find distractor for word at in

/usr/local/lib/python3.7/dist-packages/gluonnlp/data/sampler.py:354: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[18]
  str(unused_bucket_keys))


In [ ]:
for inds in ind_list:
  start_ind,end_ind=inds
  try:
   test.compare_batches(batches,fill_sent,sent_ind,3,start_ind,end_ind,filler=True)
  except ValueError:
    print(f"The following start_ind/end_ind pair raised a ValueError: {inds}")
print(fill_sent == fill_sent_copy)

The following start_ind/end_ind pair raised a ValueError: (-3, 6)
The following start_ind/end_ind pair raised a ValueError: (3, 11)
Round 1/9: Find distractor for word at index 2 by comparing 10 potential distractors
5.72
Time to evaluate >10 distractors: 0.21388554573059082
Final parameter settings: 5.819999999999999 0.2
Thomas beobachtete Weiss [seinen]
Unacceptable? 0
Thomas beobachtete sollte [seinen]
Unacceptable? 1
Thomas beobachtete hast [seinen]
Unacceptable? 1
Round 2/9: Find distractor for word at index 2 by comparing 20 potential distractors
5.72
Time to evaluate >20 distractors: 0.2118053436279297
Final parameter settings: 5.819999999999999 0.2
Distractor hast already rated 1
Thomas beobachtete warum [seinen]
Unacceptable? 0
Thomas beobachtete sowie [seinen]
Unacceptable? 0
Round 3/9: Find distractor for word at index 2 by comparing 30 potential distractors
5.72
Time to evaluate >30 distractors: 0.22357797622680664
Final parameter settings: 5.819999999999999 0.2
Thomas beob

/usr/local/lib/python3.7/dist-packages/gluonnlp/data/sampler.py:354: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[8, 9]
  str(unused_bucket_keys))


Time to evaluate >30 distractors: 0.5278100967407227
Final parameter settings: 4.71 0.2
Distractor aussagen already rated 1
Distractor erfüllt already rated 1
Distractor Weshalb already rated 1
Round 7/9: Find distractor for word at index 4 by comparing 10 potential distractors
5.11
Time to evaluate >10 distractors: 0.2130599021911621
Final parameter settings: 5.21 0.2
Thomas beobachtete seinen Nachbarn ihres, [Peter,]
Unacceptable? 1
Thomas beobachtete seinen Nachbarn hält, [Peter,]
Unacceptable? 1
Thomas beobachtete seinen Nachbarn setzen, [Peter,]
Unacceptable? 1
Round 8/9: Find distractor for word at index 4 by comparing 20 potential distractors
5.11
Time to evaluate >20 distractors: 0.21320581436157227
Final parameter settings: 5.21 0.2
Distractor ihres already rated 1
Distractor hält already rated 1
Distractor setzen already rated 1
Round 9/9: Find distractor for word at index 4 by comparing 30 potential distractors
5.11
Time to evaluate >30 distractors: 0.21512913703918457
Final